In [64]:
import requests, json
from bs4 import BeautifulSoup
from langchain_openai.chat_models.azure import AzureChatOpenAI


In [65]:
with open('config.json', 'r') as file:
    variaveis = json.load(file)


In [66]:
def extract_text_from_url(url):
    response = requests.get(url)

    if(response.status_code == 200):
        soup = BeautifulSoup(response.text, 'html.parser')
        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()
        
        text = soup.get_text(separator=' ')
        linhas = (line.strip() for line in text.splitlines())
        texto_limpo = ' '.join(linha for linha in linhas if linha)

        return texto_limpo
    else:
        print(f"Erro ao acessar pagina Status code{response.status_code}")
        return None



In [67]:
url_Text = extract_text_from_url('https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo')
print(url_Text)

Azure Open AI in VNet - DEV Community Skip to content Navigation menu Search Powered by Search Algolia Search Log in Create account DEV Community Close Add reaction Like Unicorn Exploding Head Raised Hands Fire Jump to Comments Save More... Copy link Copy link Copied to Clipboard Share to Twitter Share to LinkedIn Share to Reddit Share to Hacker News Share to Facebook Share to Mastodon Report Abuse Kenichiro Nakamura Posted on  Oct 12, 2023 Azure Open AI in VNet # azure # openai # security GPT models are hosted in multiple service vendor at the moment, and Microsoft Azure is one of them. Even though the models themselves are the same, there are many differences including: cost functionalities type of models and versions geo location security support etc. One of the most important aspects when we use it in an Enterprise Environment is, of course, security. By using Azure network security features with Azure Open AI, customers can consume the Open AI service from and within the VNet, the

In [68]:
client = AzureChatOpenAI(
    azure_endpoint = variaveis['openai_azure_endpoint'],
    api_key = variaveis['openai_api_key'],
    api_version = variaveis['open_api_version'],
    deployment_name = variaveis['open_api_deployment_name'],
    max_retries = 0
)

In [69]:
def translate_url(text, lang):
    messages = [
        ("system", "Você atua como tradutor de textos"),
        ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown" )
    ]
    response = client.invoke(messages)
    return response.content


In [70]:
url_traduzida = (translate_url(url_Text,"portugues"))

In [71]:
print(url_traduzida)

# Azure Open AI na VNet

## Kenichiro Nakamura
Publicado em 12 de outubro de 2023

### Azure Open AI na VNet

Os modelos GPT estão hospedados em vários fornecedores de serviços no momento, e o Microsoft Azure é um deles. Embora os modelos em si sejam os mesmos, existem muitas diferenças, incluindo: custo, funcionalidades, tipo de modelos e versões, localização geográfica, suporte de segurança etc. Um dos aspectos mais importantes ao usá-lo em um ambiente corporativo é, claro, a segurança. Ao usar os recursos de segurança de rede do Azure com o Azure Open AI, os clientes podem consumir o serviço Open AI de e dentro da VNet, portanto, nenhuma informação está fluindo publicamente.

### Exemplo de Implantação

O repositório de amostras do Azure fornece arquivos de bicep de exemplo para implantar o Azure Open AI em um ambiente VNet. GitHub: openai-enterprise-iac

As principais características utilizadas pelo bicep são:
- VNet
- Integração VNet para Aplicativo Web
- Endpoint Privado para o A

In [3]:
# Add your key and endpoint
key = variaveis['key']
endpoint = "https://api.cognitive.microsofttranslator.com"

location = variaveis['location']

language_destination = 'pt-br'


In [4]:
def translator_text(text, target_language):
    path = '/translate'
    constructed_url = endpoint + path

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    body = [{
        'text': text
    }]

    params = {
    'api-version': '3.0',
    'from': 'en',
    'to': target_language
    }

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()

    return response[0]['translations'][0]["text"]



In [5]:
def translator_document(path):
    document = Document(path)
    full_text = []
    for paragraph in document.paragraphs:
        full_text.append(translator_text(paragraph.text, language_destination))
    
    translated_doc = Document()
    for line in full_text:
        translated_doc.add_paragraph(line)

    path_translated = path.replace(".docx", f"_{language_destination}.docx")
    translated_doc.save(path_translated)

    return path_translated

In [6]:
translation = translator_text("hello, how are you", language_destination)
print(translation)

Olá, tudo bem


In [7]:
path_translated = translator_document("traduzir.docx")
print(path_translated)

traduzir_pt-br.docx
